In [250]:
%load_ext autoreload
%autoreload 2
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import pandas as pd
import os
import sys
import numpy as np
sys.path.append("../../Code")
import utils
import files as f
import seaborn as sns
import matplotlib.pyplot as plt
import preprocessing as pre
import languages as lang
stopWords = set(stopwords.words('english'))
import profiles
import afinns as af

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [251]:
guests = pd.read_csv(os.path.join('..','2','nr_restrict_guests.csv'), low_memory=False, index_col=0).drop_duplicates(subset="id", keep="last")
hosts = pd.read_csv(os.path.join('..','2','nr_restrict_hosts.csv'), index_col=0)
hosts = hosts[~hosts.index.isnull()]

In [252]:
len(guests)

96255

In [253]:
len(hosts)

6924

## Load Translated Profiles

In [23]:
trans_guest = pd.read_csv('translated_guest_profiles.csv', index_col=0)
trans_host = pd.read_csv('translated_host_profiles.csv', index_col=0)

In [24]:
len(trans_host)

4716

In [25]:
len(trans_guest)

52297

In [248]:
text = "I live in New York city with my wife (Amy) and two boys (Webber and Beckett). I work for an Italian wine importer, and my wife is a writer (Manifesta, Grassroots, Opting In) and founder of Soapbox, a lecture agency. Because of my work, we eat out quite frequently and are well versed with the city's restaurant scene. We are an athletic family (basketball, soccer, baseball, running) and frequently can be seen on our bikes, exploring new places in and around the city. We love to travel (Costa Rica, Sun Valley, Pacific Northwest, Italy in particular), and to be active outdoors. In fact, we loooooove to spend as little time indoors the appartment as possible! But when we are home, there's nothing wrong with hey good jigsaw puzzle should or game of backgammon by candlelight. Hope you enjoy our place!"

In [249]:
token_text = utils.tokenize(text)
pos_tags =  nltk.pos_tag(token_text)
print(pos_tags)

[('I', 'PRP'), ('live', 'VBP'), ('in', 'IN'), ('New', 'NNP'), ('York', 'NNP'), ('city', 'NN'), ('with', 'IN'), ('my', 'PRP$'), ('wife', 'NN'), ('Amy', 'NNP'), ('and', 'CC'), ('two', 'CD'), ('boys', 'NNS'), ('Webber', 'NNP'), ('and', 'CC'), ('Beckett', 'NNP'), ('I', 'PRP'), ('work', 'VBP'), ('for', 'IN'), ('an', 'DT'), ('Italian', 'JJ'), ('wine', 'NN'), ('importer', 'NN'), ('and', 'CC'), ('my', 'PRP$'), ('wife', 'NN'), ('is', 'VBZ'), ('a', 'DT'), ('writer', 'NN'), ('Manifesta', 'NNP'), ('Grassroots', 'NNP'), ('Opting', 'NNP'), ('In', 'IN'), ('and', 'CC'), ('founder', 'NN'), ('of', 'IN'), ('Soapbox', 'NNP'), ('a', 'DT'), ('lecture', 'NN'), ('agency', 'NN'), ('Because', 'IN'), ('of', 'IN'), ('my', 'PRP$'), ('work', 'NN'), ('we', 'PRP'), ('eat', 'VBP'), ('out', 'RP'), ('quite', 'RB'), ('frequently', 'RB'), ('and', 'CC'), ('are', 'VBP'), ('well', 'RB'), ('versed', 'VBN'), ('with', 'IN'), ('the', 'DT'), ("city's", 'NN'), ('restaurant', 'JJ'), ('scene', 'NN'), ('We', 'PRP'), ('are', 'VBP'), (

In [155]:
np.sign(af.get_score ('disaster'))

-1.0

In [194]:
import nltk
from nltk.tokenize import sent_tokenize
import collections as ct
def get_personality_metrics(text):
    text = str(text)
    if(len(text) == 0):
        return None
    token_text = utils.tokenize(text)
    if(len(token_text) == 0):
        return None
    sent_token_text = sent_tokenize(text)
    pos_tags =  nltk.pos_tag(token_text)
    pos_dict = dict(nltk.Index((value, key) for (key,value) in pos_tags))
    result = {}
    
    punct = ".?!("
    cd = {c:val for c, val in ct.Counter(text).items() if c in punct}

    negations = set(['no', 'not', 'none','nobody', 'nothing', 'neither', 'nowhere', 'never'])
    articles = ['DT']
    adverbs = ['RB', 'RBR','RBS']
    adjectives = ['JJ', 'JJR', 'JJS']
    verbs = ['VB', 'VBD', 'VBG', 'VBN', 'VBP','VBZ']
    nouns = ['NN', 'NNS']
    pronouns = ['NNP', 'NNPS']
    interjections = ['UH']
    prepositions = ['IN']
    when_words = ['WDT', 'WP', 'WP$', 'WRB']
    conjunctions = ['CC']
    modal_words = ['MD']
    foreign_words = ['FW']
    other_pronouns = ['PRP$']
    particles = ['RP']
    numbers  = ['CD']
    
    
    articles_freq = get_freq(articles, pos_dict)
    adverbs_freq = get_freq(adverbs, pos_dict)
    adjectives_freq = get_freq(adjectives, pos_dict)
    verbs_freq = get_freq(verbs, pos_dict)
    nouns_freq = get_freq(nouns, pos_dict)
    pronouns_freq = get_freq(pronouns, pos_dict)
    interjections_freq = get_freq(interjections, pos_dict)
    prepositions_freq = get_freq(prepositions, pos_dict)
    
    tokens_lower = [i.lower() for i in token_text]
    unique_words = set(tokens_lower)
#     emo_tokens = [np.sign(af.get_score(t)) for t in unique_words]
    pro_lower = [i.lower() for i in pos_dict['PRP']] if 'PRP' in pos_dict.keys() else []
    
    
    word_lengths = [len(w) for w in token_text]
    result['wc'] = len(token_text)
    result['avg_word_len'] = np.mean(word_lengths)
    result['avg_sent_len'] = np.mean([len(s) for s in sent_token_text])
    result['avg_long_words'] = np.sum([1 for w in word_lengths if w > 6])/len(token_text)
    result['avg_nums'] =  get_freq(numbers, pos_dict)/len(token_text)
    result['sent_count'] = len(sent_token_text)
    result['articles'] = articles_freq/len(token_text)
    result['adverbs'] = adverbs_freq/len(token_text)
    result['adjectives'] = adjectives_freq/len(token_text)
    result['verbs'] = verbs_freq/len(token_text)
    result['nouns'] = nouns_freq/len(token_text)
    result['pronouns'] = pronouns_freq/len(token_text)
    result['interjections'] = interjections_freq/len(token_text)
    result['prepositions'] = prepositions_freq/len(token_text)
    result['i'] = pro_lower.count("i")/len(token_text)
    result['we'] = pro_lower.count("we")/len(token_text)
    result['you'] = pro_lower.count("you")/len(token_text)
    result['self'] = np.sum([1 for i in pro_lower if i.endswith("self")])/len(token_text)
    result['pos_count'] = len(pos_dict.keys())
    result['unique_words'] = len(unique_words)/result['wc']
    result['negations'] = np.sum([1 for t in token_text if t.lower() in negations])/len(token_text)
    result['formality'] = (nouns_freq + adjectives_freq + prepositions_freq + articles_freq - pronouns_freq - verbs_freq - adverbs_freq - interjections_freq + 100)/2
    result['when'] = get_freq(when_words, pos_dict)/len(token_text)
    result['conj'] = get_freq(conjunctions, pos_dict)/len(token_text)
    result['foreign'] = get_freq(foreign_words, pos_dict)/len(token_text)
    result['other_pro'] = get_freq(other_pronouns, pos_dict)/len(token_text)
    result['modal'] = get_freq(modal_words, pos_dict)/len(token_text)
    result['particles'] = get_freq(particles, pos_dict)/len(token_text)
    result['capital'] = np.sum([1 for w in token_text if w.isupper()])/len(token_text)
    result['qmarks'] = cd['?'] if '?' in cd.keys() else 0
    result['brackets'] = cd['('] if '(' in cd.keys() else 0
    result['stop'] = cd['.'] if '.' in cd.keys() else 0
    result['exclaim'] = cd['!'] if '!' in cd.keys() else 0
#     result['positive_words'] = emo_tokens.count(1)/len(unique_words)
#     result['negative_words'] = emo_tokens.count(-1)/len(unique_words)

    return result

In [176]:
def get_freq(group, dic):
    res = 0
    for g in group:
        if g in dic.keys():
            res += len(dic[g])
    return res

In [195]:
personality = [get_personality_metrics(t) for t in trans_host['updated_about']]

In [196]:
trans_host['personality'] = personality

In [198]:
trans_guest['personality'] = [get_personality_metrics(t) for t in trans_guest['updated_about']]

In [200]:
trans_guest.to_csv('guests_linguistic_habits.csv')

In [201]:
trans_host.to_csv('hosts_linguistic_habits.csv')

In [208]:
trans_host

,id,lang,lang_conf,updated_about,personality
0,8758313,en,1.000000,"We are Stuart and Liubov Bragg- I, Stuart have...","{'wc': 125, 'avg_word_len': 3.816, 'avg_sent_l..."
1,4982535,en,1.000000,My boyfriend and I live in the apartment. Ther...,"{'wc': 260, 'avg_word_len': 4.3307692307692305..."
2,42133913,en,1.000000,"World traveler, just want to share the fun !","{'wc': 8, 'avg_word_len': 4.25, 'avg_sent_len'..."
3,1772076,en,1.000000,"Physician, disability rights advocate.","{'wc': 4, 'avg_word_len': 8.25, 'avg_sent_len'..."
4,39345759,en,1.000000,Hello! I am a New Yorker who loves to travel. ...,"{'wc': 37, 'avg_word_len': 4.243243243243243, ..."
5,11608565,en,1.000000,I like to travel in order to discover new plac...,"{'wc': 32, 'avg_word_len': 3.875, 'avg_sent_le..."
6,80995603,en,1.000000,"Single professional whose clean, respectful an...","{'wc': 18, 'avg_word_len': 4.888888888888889, ..."
7,78629578,en,1.000000,Hi I'm Adam and I want to welcome you to New Y...,"{'wc': 69, 'avg_word_len': 3.579710144927536, ..."
8,7138636,en,1.000000,"I travel often, love quality coffee, good book...","{'wc': 22, 'avg_word_len': 4.2727272727272725,..."
9,154766459,en,1.000000,Air BnB said I should share this about myself:...,"{'wc': 104, 'avg_word_len': 4.115384615384615,..."


In [212]:
trans_h_personality = trans_host[['id']].join(pd.DataFrame.from_records([{} if i == None else i for i in trans_host['personality']]))

In [214]:
trans_g_personality = trans_guest[['id']].join(pd.DataFrame.from_records([{} if i == None else i for i in trans_guest['personality']]))

In [218]:
g_reviews = reviews[['id', 'reviewer_id']].join(trans_g_personality.set_index('id'), on ='reviewer_id')

In [220]:
h_reviews = reviews[['id', 'recipient_id']].join(trans_h_personality.set_index('id'), on ='recipient_id')

In [223]:
cols = set(h_reviews.columns) - set(['id', 'recipient_id', 'reviewer_id'])

In [225]:
result = pd.DataFrame({'id': list(reviews.id)})

In [236]:
len(result)

215960

In [237]:
for c in cols: 
    result.is_copy = False
    result[str(c)] = h_reviews[str(c)] - g_reviews[str(c)]

In [240]:
result[~result.sent_count.isnull()].to_csv('profile_linguistic.csv')

## Get Reviews 

In [215]:
reviews = pd.read_csv(os.path.join('..','2','nr_restrict_reviews.csv'), index_col=0)

In [402]:
hg_profiles = utils.merge_two_dicts(host_empath, guest_empath)

In [403]:
relevant_reviews = reviews[(reviews['recipient_id'].isin(hg_profiles.keys())) & (reviews['reviewer_id'].isin(hg_profiles.keys()))]